In [14]:
import gc
import os
import warnings

warnings.filterwarnings("ignore")
import itertools
import random

import joblib
import numpy as np
import pandas as pd
import scipy as sp

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import gc
import warnings
from itertools import combinations

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import scipy as sp

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import itertools
from hyperopt import hp, fmin, tpe
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from hyperopt import hp, fmin, tpe
#from numpy.random import RandomState
from sklearn.metrics import mean_squared_error

In [2]:
# ====================================================
# Read data
# ==================================================== 
def read_data():
    train = pd.read_parquet("./train_fe_plus_plus.parquet")
    test = pd.read_parquet("./test_fe_plus_plus.parquet")
    return train, test


In [3]:
train,test = read_data()

In [15]:
def params_append(params):
    """
    动态回调参数函数，params视作字典
    :param params:lgb参数字典
    :return params:修正后的lgb参数字典
    """
    params['feature_pre_filter'] = False
    params['objective'] = 'binary'
    params['metric'] = 'binary_logloss'
    #params['bagging_seed'] = 2020
    return params


In [19]:
def param_hyperopt(train):
    """
    模型参数搜索与优化函数
    :param train:训练数据集
    :return params_best:lgb最优参数
    """
    # Part 1.划分特征名称，删除ID列和标签列
    label = 'target'
    features = train.columns.tolist()
    features.remove('customer_ID')
    features.remove('target')
    
    # Part 2.封装训练数据
    train_data = lgb.Dataset(train[features], train[label])
    
    # Part 3.内部函数，输入模型超参数损失值输出函数
    def hyperopt_objective(params):
        """
        输入超参数，输出对应损失值
        :param params:
        :return:最小rmse
        """
        # 创建参数集
        params = params_append(params)
        print(params)
        
        # 借助lgb的cv过程，输出某一组超参数下损失值的最小值
        res = lgb.cv(params, train_data, 1000,
                     nfold=2,
                     stratified=False,
                     shuffle=True,
                     metrics='rmse',
                     early_stopping_rounds=20,
                     verbose_eval=False,
                     show_stdv=False,
                     seed=2020)
        return min(res['rmse-mean']) # res是个字典

    # Part 4.lgb超参数空间
    params_space = {
        'learning_rate': hp.uniform('learning_rate', 1e-2, 5e-1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
        'num_leaves': hp.choice('num_leaves', list(range(10, 300, 10))),
        'reg_alpha': hp.randint('reg_alpha', 0, 10),
        'reg_lambda': hp.uniform('reg_lambda', 0, 10),
        'bagging_freq': hp.randint('bagging_freq', 1, 10),
        'min_child_samples': hp.choice('min_child_samples', list(range(1, 30, 5)))
    }
    
    
    # Part 5.TPE超参数搜索
    params_best = fmin(
        hyperopt_objective,
        space=params_space,
        algo=tpe.suggest,
        max_evals=30,
        rstate=np.random.default_rng(2020))
        #rstate=RandomState(2020))
    
    # 返回最佳参数
    return params_best

In [ ]:
best_clf = param_hyperopt(train)

{'bagging_fraction': 0.9429104308567877, 'bagging_freq': 2, 'feature_fraction': 0.5715782198140802, 'learning_rate': 0.21315219327595428, 'min_child_samples': 11, 'num_leaves': 160, 'reg_alpha': 3, 'reg_lambda': 7.561160634893758, 'feature_pre_filter': False, 'objective': 'binary', 'metric': 'binary_logloss'}
[LightGBM] [Info] Number of positive: 59390, number of negative: 170066                                                
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.491737 seconds.             
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255446                                                                                    
[LightGBM] [Info] Number of data points in the train set: 229456, number of used features: 1636                        
[LightGBM] [Info] Number of positive: 59437, number of negative: 170019                                                
[LightGBM] [Warning] Auto-choos

[LightGBM] [Info] Number of data points in the train set: 229456, number of used features: 1636                        
[LightGBM] [Info] Number of positive: 59437, number of negative: 170019                                                
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.944444 seconds.             
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255446                                                                                    
[LightGBM] [Info] Number of data points in the train set: 229456, number of used features: 1636                        
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258830 -> initscore=-1.052061                                        
[LightGBM] [Info] Start training from score -1.052061                                                                  
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259034 -> initscore=-1.050993                       

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258830 -> initscore=-1.052061                                        
[LightGBM] [Info] Start training from score -1.052061                                                                  
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259034 -> initscore=-1.050993                                        
[LightGBM] [Info] Start training from score -1.050993                                                                  
{'bagging_fraction': 0.9827696256425138, 'bagging_freq': 6, 'feature_fraction': 0.6241629598983252, 'learning_rate': 0.2693431830624993, 'min_child_samples': 11, 'num_leaves': 30, 'reg_alpha': 7, 'reg_lambda': 5.653589408911715, 'feature_pre_filter': False, 'objective': 'binary', 'metric': 'binary_logloss'}
[LightGBM] [Info] Number of positive: 59390, number of negative: 170066                                                
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.

In [ ]:
# ====================================================
# Library
# ====================================================
import gc
import os
import warnings

warnings.filterwarnings("ignore")
import itertools
import random

import joblib
import numpy as np
import pandas as pd
import scipy as sp

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import gc
import warnings
from itertools import combinations

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import scipy as sp

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import itertools

from tqdm.auto import tqdm

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = "./"
    seed = 53
    n_folds = 5
    target = "target"
    boosting_type = "53-5-88000-22000-0.003"


# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)


# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + "train_diff.parquet")
    test = pd.read_parquet(CFG.input_dir + "test_diff.parquet")
    return train, test


# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:, 0] == 0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:, 0]) / np.sum(labels[:, 0])
    gini = [0, 0]
    for i in [1, 0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:, 0] == 0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] * weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1] / gini[0] + top_four)


# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return "amex_metric", amex_metric(y_true, y_pred), True


import pickle

class DartEarlyStopping(object):
    def __init__(self, data_name, monitor_metric, stopping_round):
        self.data_name = data_name
        self.monitor_metric = monitor_metric
        self.stopping_round = stopping_round
        self.best_score = None
        self.best_model = None
        self.best_score_list = []
        self.best_iter = 0

    def _is_higher_score(self, metric_score, is_higher_better):
        if self.best_score is None:
            return True
        return (self.best_score < metric_score) if is_higher_better else (self.best_score > metric_score)

    def _deepcopy(self, x):
        return pickle.loads(pickle.dumps(x))

    def __call__(self, env):
        evals = env.evaluation_result_list
        for data, metric, score, is_higher_better in evals:
            if data != self.data_name or metric != self.monitor_metric:
                continue
            if not self._is_higher_score(score, is_higher_better):
                if env.iteration - self.best_iter > self.stopping_round:
                    eval_result_str = '\t'.join([lgb.callback._format_eval_result(x) for x in self.best_score_list])
                    print(f"Early stopping, best iteration is:\n[{self.best_iter+1}]\t{eval_result_str}") 
                    print(f"You can get best model by \"DartEarlyStopping.best_model\"")
                    raise lgb.callback.EarlyStopException(self.best_iter, self.best_score_list)
                return

            self.best_model = self._deepcopy(env.model)
            self.best_iter = env.iteration
            self.best_score_list = evals
            self.best_score = score
            return
        raise ValueError("monitoring metric not found")



# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(
        train.dtypes[(train.dtypes == "float32") | (train.dtypes == "float64")].index
    )
    num_cols = [col for col in num_cols if "last" in col]
    for col in num_cols:
        train[col + "_round2"] = train[col].round(2)
        test[col + "_round2"] = test[col].round(2)
    
    
    
    # Get the difference between last and mean
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
    # Transform float64 and float32 to float16
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)
        
        
    # Get feature list
    features = [col for col in train.columns if col not in ["customer_ID", CFG.target]]
    params = {
        'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.003,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40
        }
    
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(" ")
        print("-" * 50)
        print(f"Training fold {fold} with {len(features)} features...")
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
#        y_train, y_val = (
#            train[CFG.target].iloc[trn_ind],
#            train[CFG.target].iloc[val_ind],
#        )
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)

        es = DartEarlyStopping("valid_0", "amex_metric", stopping_round=22000)
        model = lgb.train(
            params=params,
            train_set=lgb_train,
            num_boost_round=88000,
            valid_sets=[lgb_valid],
            early_stopping_rounds=22000,
            callbacks=[es],
            verbose_eval=50,
            feval=lgb_amex_metric,
        )

        # Save best model
        joblib.dump(
            model,
            f"lgbm_fold{fold}_{CFG.boosting_type}_seed{CFG.seed}.pkl",
        )
        # Predict validation
        print("ready to Predict validation")
        val_pred = model.predict(x_val)
        # Add to out of folds array
        print("ready to Add to out of folds array")
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        print("ready to Predict the test set")
        test_pred = model.predict(test[features])
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f"Our fold {fold} CV score is {score}")
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f"Our out of folds CV score is {score}")
    # Create a dataframe to store out of folds predictions
#    oof_df = pd.DataFrame(
#        {
#            "customer_ID": train["customer_ID"],
#            "target": train[CFG.target],
#            "prediction": oof_predictions,
#        }
#    )
#    oof_df.to_csv(
#        f"oof_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv",
#        index=False,
#    )
    # Create a dataframe to store test prediction
#    test_df = pd.DataFrame(
#        {"customer_ID": test["customer_ID"], "prediction": test_predictions}
#    )
#    test_df.to_csv(
#        f"test_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv",
#        index=False,
#    )

    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(f'modifioof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    # Create a dataframe to store test prediction
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(f'modifitest_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


seed_everything(CFG.seed)
train, test = read_data()
train_and_evaluate(train, test)